In [30]:
from gensim.test.utils import common_texts,get_tmpfile
from gensim.models import word2vec
from gensim import corpora


from nltk.corpus import reuters,stopwords
from nltk.tokenize import sent_tokenize,word_tokenize,RegexpTokenizer

from nltk import Text

import numpy as np
import pandas as pd
import re,string

# from pprint import pprint

In [97]:
def gen_text():
    '''        
    generate text from corpus
    
    input: corpus ID
    return: list, each for one text
    '''

    corp_select = reuters
    id_ls =  corp_select.fileids()
    text_nums = 4000

    text_ls = []
    for k in range(text_nums):
        text_ls.append(corp_select.raw(fileids=id_ls[k]))

    return text_ls



In [98]:
def gen_sentence(text_ls):
    '''
    generate tokenized sentences from text list
    
    input: corpus ID
    return: list, each for one sentence
    '''

    sen_ls = []
    for text in text_ls:
        sen_ls = sen_ls + sent_tokenize(text)

    return sen_ls

In [99]:
def remove_punc(word):
    ''' 去除标点符号 remove punctuation from word'''

    trans=str.maketrans({key: None for key in string.punctuation}) #建立转换关系
    word = word.translate(trans)

    return word

In [100]:
def gen_word(sentence):
    '''
    generate tokenized words from sentence
    remove punctuation

    input: sentence
    return: list, each element for one word
    '''

    word_ls = []
    reg_tokenizer_b = RegexpTokenizer(r'\s+',gaps=True) ## 按照空格划分
    reg_tokenizer_w = RegexpTokenizer(r'\w+') ## 按照单词

    
        # word_ls.append(word_tokenize(sen))

    word_ls = reg_tokenizer_b.tokenize(sentence)
    # word_ls = list(filter(lambda x:x not in stopwords.words('english'),word_ls))
    # word_ls = list(filter(lambda x:x not in [*map(lambda x:x.upper(),stopwords.words('english'))],word_ls))

    word_ls = list(map(lambda x:re.sub("[0-9]+","",x),word_ls))
    word_ls = list(map(lambda x:x.lower(),word_ls))
        # word_ls_tmp = list(map(lambda x:x.translate(trans),word_ls_tmp))

    word_ls = list(map(remove_punc,word_ls))
    word_ls = list(filter(lambda x:len(x)>0,word_ls)) ## 去掉空格 

    return word_ls


In [101]:
text_ls = gen_text()
sen_ls = gen_sentence(text_ls)

len(text_ls),len(sen_ls)



(4000, 19520)

In [102]:
word_sen_ls = []
word_ls = []

for xx in sen_ls:
    word_sen_ls.append(gen_word(xx))
    word_ls = word_ls + gen_word(xx)

In [120]:
vocab = np.unique(word_ls)
print(len(text_ls),len(word_sen_ls),len(word_ls),len(vocab))

4000 19520 468094 20678


In [121]:
word_ls = pd.Series(word_ls)
is_common_word = word_ls.value_counts()>=50

In [122]:
vocab = is_common_word[is_common_word==True].index.tolist()
len(vocab)

1115

In [123]:
wv_model = word2vec.Word2Vec(word_sen_ls,size=100,window=5)

In [124]:
model_vocab = list(wv_model.wv.vocab.keys())
len(model_vocab)

6215

In [125]:
df_word_vec = pd.DataFrame(index=vocab,columns=['vec_'+str(i) for i in range(100)])

In [127]:
for i,word in enumerate(df_word_vec.index[:]):
    # if i%100==0:
    #     print(i)
    df_word_vec.loc[word] = wv_model.wv.word_vec(word).round(4)

In [128]:
df_word_vec.head()

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_90,vec_91,vec_92,vec_93,vec_94,vec_95,vec_96,vec_97,vec_98,vec_99
the,-0.1518,0.4571,-0.0052,0.2874,-0.6003,-0.5068,0.4788,0.0543,0.0351,1.3316,...,-0.0342,-0.4633,-0.2218,1.0928,0.0449,-0.1377,0.5191,-0.0101,0.5749,0.1787
of,-0.0841,0.4754,0.5144,0.0162,-0.2329,-0.2922,0.0424,-0.0125,0.3198,-0.185,...,-1.0748,0.187,-0.443,0.5509,0.734,0.1633,-0.5504,-0.0323,-0.0636,0.5541
to,-0.547,0.2666,-0.1222,-0.0656,-1.0751,0.4433,0.4562,1.2213,0.7926,-0.1983,...,-0.4325,0.4254,-0.5346,0.2084,0.4553,-0.6684,0.4974,-0.1434,0.8088,-0.3856
in,-0.0899,0.3626,0.0153,0.0658,0.0789,-0.0363,0.1631,0.0857,0.8244,-0.3604,...,-0.8285,0.6386,-0.4927,0.5591,0.0553,-0.0838,-0.7433,0.0274,0.3288,0.1136
and,-0.2367,0.0411,-0.3654,-0.3301,-0.5453,-0.0321,0.1703,0.0685,0.2182,0.0282,...,-0.4153,0.2055,-0.0755,0.2019,0.0095,-0.2883,-0.8199,-0.3857,-0.3366,0.1847


In [129]:
# df_

In [130]:
wv_model.wv.word_vec('the')

array([-0.15181644,  0.45714858, -0.00524186,  0.28742513, -0.60031974,
       -0.5067548 ,  0.47876546,  0.05425365,  0.03506719,  1.3316201 ,
        0.76646566, -0.6048728 , -0.5493429 , -0.3991539 ,  0.94563085,
        0.22750299,  0.412729  , -0.64330274, -0.29419848, -1.1044321 ,
        0.3881118 ,  0.66417676, -0.44369525, -0.9205933 ,  0.40129432,
       -1.0497384 ,  0.23863624,  1.1829684 ,  0.19989215,  0.00802428,
        0.33290794,  0.24115509,  0.48210523,  0.45008874, -1.0136256 ,
        0.25461698,  0.47712317, -0.27585095, -0.39312762,  0.7589427 ,
        0.16635601,  0.03966455,  0.0150586 , -0.11332312, -1.6524109 ,
       -0.41302818,  0.43863103, -1.2761809 , -0.06216665, -0.22265533,
       -0.7991344 , -0.17343867,  0.5568801 , -0.5604664 ,  0.4138363 ,
        0.24328394,  0.09199163,  0.25972098, -1.1394298 ,  0.7562514 ,
        0.6668302 ,  0.18704392, -0.4377473 , -0.09302838,  0.1436386 ,
        0.13761836, -0.26423758, -0.3485164 ,  0.27410793,  0.72

In [12]:
sent_tokenize(EXAMPLE_TEXT)

['Hello Mr. Smith, how are you doing today?',
 'The weather is great, and Python is awesome.',
 'The sky is pinkish-blue.',
 "You shouldn't eat cardboard."]

In [31]:
# import nltk
# nltk.download('punkt')

FileSystemPathPointer('C:\\Users\\qqmai\\nltk_data')

In [29]:
corpora.textcorpus.TextCorpus.getstream()

TypeError: getstream() missing 1 required positional argument: 'self'